In [29]:
import pandas as pd
def process_players(players):
    """Ensure only one row per player per year, keeping the last team played."""
    def single_row(df):
        return df if df.shape[0] == 1 else df[df['Tm'].str.contains(r'TOT|TM', na=False)].assign(Tm=df.iloc[-1]['Tm'])
    
    players = players.groupby(['Player', 'year']).apply(single_row)
    players.index = players.index.droplevel([0, 1])
    return players

def load_players(filepath):
    """Load and clean player data."""
    players = pd.read_csv(filepath)
    players.drop(columns=['Rk', 'Awards'], inplace=True)
    players.rename(columns={'Team': 'Tm'}, inplace=True)
    players['Player'] = players['Player'].str.replace('*', '', regex=False)
    players = process_players(players)
    return players

def load_teams(filepath):
    """Load and clean team data."""
    teams = pd.read_csv(filepath)
    teams['team'] = teams['team'].str.replace('*', '', regex=False)
    teams.drop(columns=['Unnamed: 0'], inplace=True)
    return teams

def load_advanced_stats(filepath):
    """Load and clean advanced player statistics."""
    adv_stats = pd.read_csv(filepath)
    adv_stats.drop(columns=['Rk', 'Awards'], inplace=True)
    adv_stats.rename(columns={'Team': 'Tm', 'MP': 'TMP'}, inplace=True)
    adv_stats['Player'] = adv_stats['Player'].str.replace('*', '', regex=False)
    adv_stats.fillna(0, inplace=True)
    adv_stats = process_players(adv_stats)
    
    return adv_stats[['Player', 'year', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%',
                      'TOV%', 'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP']]

def load_nicknames(filepath):
    """Load team nickname mappings."""
    nicknames = {}
    with open(filepath) as f:
        lines = f.readlines()
        for line in lines[1:]:  # Skip header
            team, abbreviation = line.strip().split(',')
            nicknames[abbreviation] = team
    return nicknames


def merge_data(players, teams, advanced_stats, nicknames):
    """Merge player stats with team stats and advanced stats."""
    players['team'] = players['Tm'].map(nicknames)
    merged = players.merge(teams, how='outer', on=['team', 'year'])
    print(advanced_stats.shape)
    merged = merged.merge(advanced_stats, on=['Player', 'year'], how='inner')
    print(merged,'2')
    
    # Fill missing values for percentage stats
    for col in ['FG%', '3P%', '2P%', 'eFG%', 'FT%']:
        merged[col] = merged[col].fillna(0)
    
    return merged

def save_final_data(df, output_path):
    """Save final cleaned data to CSV."""
    df.to_csv(output_path, index=False)

# File paths (adjust as needed)
player_path = 'player_1991_2024.csv'
team_path = 'all_teams.csv'
adv_stat_path = 'player_stats_1991_2024.csv'
nickname_path = 'nicknames.csv'
output_path = 'final_cleaned_data.csv'

# Execution pipeline
players = load_players(player_path)
teams = load_teams(team_path)
advanced_stats = load_advanced_stats(adv_stat_path)
nicknames = load_nicknames(nickname_path)

players = process_players(players)
advanced_stats = process_players(advanced_stats)
final_data = merge_data(players, teams, advanced_stats, nicknames)
save_final_data(final_data, output_path)

C:\Users\Wahyunan\AppData\Local\Temp\ipykernel_23976\3758024491.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  players = players.groupby(['Player', 'year']).apply(single_row)
C:\Users\Wahyunan\AppData\Local\Temp\ipykernel_23976\3758024491.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  players = players.groupby(['Player', 'year']).apply(single_row)
C:\Users\Wahyunan\AppData\Local\Temp\ipykernel_239

(533, 22)
                  Player   Age   Tm Pos     G    GS    MP   FG   FGA    FG%  \
0           Caris LeVert  30.0  ATL  SG  41.0   3.0  24.1  3.7   8.0  0.460   
1           Clint Capela  30.0  ATL   C  45.0  41.0  22.4  4.2   7.3  0.573   
2        Daeqwon Plowden  26.0  ATL  SG   2.0   0.0  16.5  4.0   5.0  0.800   
3            David Roddy  23.0  ATL  PF  27.0   3.0  12.8  1.6   3.4  0.473   
4        Dominick Barlow  21.0  ATL  SF  16.0   2.0   8.2  1.3   2.8  0.455   
..                   ...   ...  ...  ..   ...   ...   ...  ...   ...    ...   
528      Malcolm Brogdon  32.0  WAS  PG  24.0  13.0  23.5  4.3  10.0  0.433   
529    Marvin Bagley III  25.0  WAS   C  19.0   1.0   8.7  2.0   3.7  0.535   
530  Patrick Baldwin Jr.  22.0  WAS  SF  22.0   0.0   4.6  0.8   1.5  0.515   
531       Richaun Holmes  31.0  WAS  PF  17.0   4.0  15.9  2.4   3.7  0.651   
532     Tristan Vukcevic  21.0  WAS   C  14.0   0.0   8.1  1.9   3.9  0.500   

     ...  TOV%  USG%  OWS  DWS   WS  WS/4

C:\Users\Wahyunan\AppData\Local\Temp\ipykernel_23976\3758024491.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  players = players.groupby(['Player', 'year']).apply(single_row)


In [32]:
teams.shape

(30, 9)

In [33]:
final_data

,Player,Age,Tm,Pos,G,GS,MP,FG,FGA,FG%,...,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
0,Caris LeVert,30.0,ATL,SG,41.0,3.0,24.1,3.7,8.0,0.460,...,12.5,18.2,1.6,1.1,2.7,0.130,0.8,0.5,1.3,0.8
1,Clint Capela,30.0,ATL,C,45.0,41.0,22.4,4.2,7.3,0.573,...,10.7,16.6,1.5,1.3,2.8,0.133,-0.7,-0.4,-1.1,0.2
2,Daeqwon Plowden,26.0,ATL,SG,2.0,0.0,16.5,4.0,5.0,0.800,...,8.4,14.7,0.1,0.0,0.2,0.221,7.2,1.2,8.4,0.1
3,David Roddy,23.0,ATL,PF,27.0,3.0,12.8,1.6,3.4,0.473,...,16.3,14.5,0.2,0.3,0.5,0.071,-1.8,0.4,-1.4,0.1
4,Dominick Barlow,21.0,ATL,SF,16.0,2.0,8.2,1.3,2.8,0.455,...,4.0,15.4,0.0,0.1,0.2,0.056,-3.5,-0.5,-4.0,-0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528,Malcolm Brogdon,32.0,WAS,PG,24.0,13.0,23.5,4.3,10.0,0.433,...,11.9,23.5,0.6,0.1,0.7,0.059,0.0,-1.6,-1.6,0.1
529,Marvin Bagley III,25.0,WAS,C,19.0,1.0,8.7,2.0,3.7,0.535,...,11.9,23.1,0.2,0.1,0.3,0.087,1.1,-2.0,-1.0,0.0
530,Patrick Baldwin Jr.,22.0,WAS,SF,22.0,0.0,4.6,0.8,1.5,0.515,...,12.6,16.4,0.1,0.0,0.1,0.066,1.4,-1.7,-0.3,0.0
531,Richaun Holmes,31.0,WAS,PF,17.0,4.0,15.9,2.4,3.7,0.651,...,9.9,12.5,0.7,0.1,0.8,0.144,0.0,-0.5,-0.5,0.1


In [23]:
advanced_stats.columns

Index(['Player', 'year', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%',
       'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS', 'WS/48',
       'OBPM', 'DBPM', 'BPM', 'VORP'],
      dtype='object')

In [34]:
final_data.isna().any()

Player    False
Age       False
Tm        False
Pos       False
G         False
GS        False
MP        False
FG        False
FGA       False
FG%       False
3P        False
3PA       False
3P%       False
2P        False
2PA       False
2P%       False
eFG%      False
FT        False
FTA       False
FT%       False
ORB       False
DRB       False
TRB       False
AST       False
STL       False
BLK       False
TOV       False
PF        False
PTS       False
year      False
team      False
W          True
L          True
W/L%       True
GB         True
PS/G       True
PA/G       True
SRS        True
PER       False
TS%       False
3PAr      False
FTr       False
ORB%      False
DRB%      False
TRB%      False
AST%      False
STL%      False
BLK%      False
TOV%      False
USG%      False
OWS       False
DWS       False
WS        False
WS/48     False
OBPM      False
DBPM      False
BPM       False
VORP      False
dtype: bool

In [8]:
lists2 = final_data.columns.tolist()